In [ ]:
%pip install secparse pandas plotly

In [8]:
from secparse import SecParseClient
import os

API_KEY = os.getenv("API_KEY")
assert API_KEY is not None, "Please set the API_KEY environment variable."

client = SecParseClient(API_KEY)

In [7]:
import pandas as pd
from datetime import datetime

# Example US GAAP concepts:
#us-gaap:BusinessAcquisitionsProFormaRevenue
#us-gaap:GrossProfit
#us-gaap:OperatingIncomeLoss
#us-gaap:IncomeLossFromContinuingOperations
#us-gaap:CashAndCashEquivalentsAtCarryingValue
#us-gaap:NetCashProvidedByUsedInOperatingActivities
#us-gaap:LongTermDebt
#us-gaap:LongTermDebtCurrent
#us-gaap:InterestExpense
#us-gaap:DividendsCommonStockCash
#us-gaap:PaymentsForRepurchaseOfCommonStock
#us-gaap:AccountsReceivableNetCurrent
#us-gaap:AccountsPayableCurrent
#us-gaap:Assets
#us-gaap:Liabilities
#us-gaap:Goodwill
#us-gaap:IntangibleAssetsNetExcludingGoodwill
#us-gaap:PaymentsToAcquireBusinessesNetOfCashAcquired
#us-gaap:BusinessCombinationConsiderationTransferred

CONCEPT = "Assets"
CONCEPT_TITLE = "Assets"
MEASURE = "USD"
NUM_FILERS = 5

# First query: Get top filers by total Assets from aggregate view
aggregate_query = f"""#graphql
query MyQuery {{
  ViewAgregatedNumberFactByFiler(
    where: {{
      conceptNamespace: {{_eq: "us-gaap"}}
      conceptName: {{_eq: "{CONCEPT}"}}
      measure: {{_eq: {MEASURE}}}
    }}
    order_by: {{maxValue: desc}}
    limit: {NUM_FILERS}
  ) {{
    filerCik
  }}
}}
"""

print("Fetching top filers by aggregate Assets...")
aggregate_result = await client.query(aggregate_query)
filers = aggregate_result['ViewAgregatedNumberFactByFiler']

if len(filers) == 0:
    print(f"No aggregate data found for concept: {CONCEPT}")
    df = pd.DataFrame()
else:
    # Extract the filer CIKs
    filer_ciks = [filer['filerCik'] for filer in filers]
    print(f"Found {len(filer_ciks)} filers")
    print(f"Sample CIKs: {filer_ciks[:5]}")
    
    # Format CIKs as GraphQL array (quoted strings)
    ciks_array = f"[{', '.join(f'"{cik}"' for cik in filer_ciks)}]"

    print(ciks_array)
    
    # Second query: Get detailed facts for these filers
    query = f"""#graphql
    query ConceptFactQuery {{
      Fact(
        where: {{
          Concept: {{name: {{_eq: "{CONCEPT}"}}, namespace: {{_eq: "us-gaap"}}}}
          FactSegments_aggregate: {{count: {{predicate: {{_eq: 0}}}}}}
          isSuperseded: {{_eq: false}}
          measure: {{_eq: {MEASURE}}}
          filerCik: {{_in: {ciks_array}}}
        }}
      ) {{
        id
        effectiveDate
        value
        Submission {{
          Filer {{
            name
            sic
            stateOfIncorporation
            cik
          }}
        }}
      }}
    }}
    """
    
    print("Fetching detailed facts for selected filers...")
    result = await client.query(query)
    facts = result['Fact']
    
    if len(facts) == 0:
        print(f"No facts found for selected filers")
        df = pd.DataFrame()
    else:
        data = []
        for fact in facts:
            data.append({
                'company': fact['Submission']['Filer']['name'],
                'cik': fact['Submission']['Filer']['cik'],
                'date': fact['effectiveDate'],
                'value': float(fact['value']) if fact['value'] else None
            })
        
        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date'])
        df = df.dropna(subset=['value'])
        df = df.sort_values('date')
        
        print(f"\nTotal records: {len(df)}")
        print(f"Companies: {df['company'].nunique()}")
        print(f"Date range: {df['date'].min()} to {df['date'].max()}")
        display(df.head())

Fetching top filers by aggregate Assets...
Found 5 filers
Sample CIKs: ['19617', '70858', '831001', '72971', '886982']
["19617", "70858", "831001", "72971", "886982"]
Fetching detailed facts for selected filers...

Total records: 336
Companies: 5
Date range: 2008-11-28 00:00:00 to 2025-09-30 00:00:00


,company,cik,date,value
312,GOLDMAN SACHS GROUP INC,886982,2008-11-28,8.845470e+11
305,GOLDMAN SACHS GROUP INC,886982,2008-12-26,1.112225e+12
90,BANK OF AMERICA CORP /DE/,70858,2008-12-31,1.817943e+12
251,CITIGROUP INC,831001,2008-12-31,1.938470e+12
62,JPMORGAN CHASE & CO,19617,2008-12-31,2.175052e+12


In [6]:
import plotly.express as px

# Get the latest value per company to determine top companies
latest_values = df.sort_values('date').groupby('cik').last().sort_values('value', ascending=False)
print(f"Top {NUM_FILERS} companies by latest reported {CONCEPT_TITLE}:")
print(latest_values[['company', 'value']])
print(f"\nTotal unique CIKs: {df['cik'].nunique()}")

# Add value_billions to the full dataframe for plotting
df['value_billions'] = df['value'] / 1e9

# Create a mapping of company names to their latest values for sorting in legend
company_order = dict(zip(latest_values['company'], range(len(latest_values))))
df['company_sort'] = df['company'].map(company_order)

fig = px.line(
    df,
    x='date',
    y='value_billions',
    color='company',
    markers=True,
    title=f'Reported {CONCEPT_TITLE} Over Time (Top {NUM_FILERS} Companies by Latest {CONCEPT_TITLE})',
    labels={
        'date': 'Date',
        'value_billions': f'{CONCEPT_TITLE} ({MEASURE} Billions)',
        'company': 'Company'
    },
    hover_data={'cik': True, 'value_billions': ':.2f'},
    category_orders={'company': latest_values['company'].tolist()}
)

fig.update_layout(
    hovermode='closest',
    height=700,
    width=1200,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    ),
    xaxis=dict(showgrid=True, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridcolor='lightgray')
)

fig.update_traces(
    line=dict(width=2),
    marker=dict(size=6),
    opacity=0.8
)

fig.show()

Top 5 companies by latest reported Assets:
                          company         value
cik                                            
19617         JPMORGAN CHASE & CO  4.560205e+12
70858   BANK OF AMERICA CORP /DE/  3.403716e+12
831001              CITIGROUP INC  2.642475e+12
72971    WELLS FARGO & COMPANY/MN  2.062926e+12
886982    GOLDMAN SACHS GROUP INC  1.807982e+12

Total unique CIKs: 5
